In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from netCDF4 import Dataset
import cartopy.crs as ccrs
import imageio

In [2]:
def actual_time(filename):
    return int(filename.split('-')[0]) + int(filename.split('-')[1].split('.')[0]) * 12

def get_date(filename):
    time_segment = filename.split('.')[-2]
    
    year = int(time_segment[:4])
    month = int(time_segment[4:])
    
    return year, month

def make_map(overlay_filename):
    map_skew = '../input/mapskewed3/map-skewed3.png'

    background = Image.open(map_skew)
    overlay = Image.open(overlay_filename)

    background = background.convert("RGBA")
    overlay = overlay.convert("RGBA")

    background.paste(overlay, (0, 0), overlay)
    
    merged_filename = overlay_filename.replace(".png", "-overlay.png")
    background.save(merged_filename,"PNG")
    
    return merged_filename

def generate_3d_overlay(filename):
    
    data = Dataset(filename,more="r")
    
    lng = data.variables["lon"][:]
    lat = data.variables["lat"][:]
    to3 = data.variables["TO3"][:,:,:]
    
    X, Y = np.meshgrid(lng, lat)
    fig = plt.figure(figsize=(30, 30))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X, Y, to3[0], rstride=1, cstride=1, cmap='coolwarm')

    ax.contour3D(X, Y, to3[0], 50, cmap='binary')

    ax.view_init(20, 340)
    ax.set_axis_off()

    year, day = get_date(filename)
    ax.set_title(f"{day}/{year}", fontdict={'fontsize': 35}, y=0.89, color="#ffffff")
    
    img_filename = f'{day}-{year}.png'
    plt.savefig(img_filename, transparent=True)
    
    return img_filename

In [ ]:
nc4dir = '../input/earthdata-merra2-co/'
all_merged = []
for file in os.listdir(nc4dir):
    img_filename = generate_3d_overlay(nc4dir + file)
    
    merged_filename = make_map(img_filename)
    all_merged.append(merged_filename)

In [ ]:
images = []
for filename in sorted(all_merged, key=lambda f: actual_time(f)):
    images.append(imageio.imread(filename))
imageio.mimsave('ozone.gif', images, duration=0.7)